# Match Meise Collectors to Wikidata Items Using *Cosine Similarity*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …
- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv(
    "data/wikidata_persons_botanists_20231030_1539.csv",
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }
)
pprint.pprint(wikidata.columns)
display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names

wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)

# cols = wd_matchtest.columns.tolist()

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(Lindberg), H.v.R.",1
2,"(Magistad), I.K.",1
3,"(Tsybulskaya), M.P.N.",1
4,"(botaniker), B.P.",1
...,...,...
65517,"Яковлевич, С.В.",1
65518,"Яфрэмаў, А.Л.",1
65519,"ас-Сури, И.",1


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(Lindberg), Hildur von Rettig",1
2,"(Magistad), Inger Kaasa",1
3,"(Tsybulskaya), Maria Pavlovna Nagibina",1
4,"(botaniker), Bror Pettersson",1
...,...,...
68027,"Яковлевич, Скробишевский, Владислав",1
68028,"Яфрэмаў, Аляксандр Лаўрэнцьевіч",1
68029,"ас-Сури, Ибн",1


## Load Collectors Data Set

**Data sources:**

- Jupyter Notebook for [create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

collectors = pd.read_csv(
    "data/Meise_doi-10.15468-dl.ax9zkh/occurrence_recordedBy_eventDate_occurrenceIDs_20230830_parsed.tsv",
    sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime or using pd.Periode(…)
print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='S' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
43651,A,H. M. Llo,NaN,NaN,NaN,NaN,NaN,NaN,H.M. L[lo][???]a,parsed:H.M.Llo a,cleaned:H. M. Llo A,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
44231,A,Hameln,NaN,NaN,NaN,NaN,NaN,NaN,Hameln [a/d Weser],parsed:Hameln a<SEP>d Weser,cleaned:Hameln A<SEP>D Weser,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
84575,A,R.,NaN,NaN,NaN,NaN,NaN,NaN,R. Decha[] & R. []a,parsed:R. Decha<SEP>R. a,cleaned:R. Decha<SEP>R. A,1,http://www.botanicalcollections.be/specimen/BR...,1988-11-19 00:00:00,1988-11-19,1988-11-19
18440,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Church, A.C., Ismail, Ruskandi, A",parsed:A.C. Church<SEP>Ruskandi Ismail<SEP>A,cleaned:A.C. Church<SEP>Ruskandi Ismail<SEP>A,1,http://www.botanicalcollections.be/specimen/BR...,1995-10-26 00:00:00,1995-10-26,1995-10-26
87779,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raynal R. & A,parsed:R. Raynal<SEP>A,cleaned:R. Raynal<SEP>A,1,http://www.botanicalcollections.be/specimen/BR...,1978-06-08 00:00:00,1978-06-08,1978-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58982,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Klášterský & Žertová,parsed:Klášterský<SEP>Žertová,cleaned:Klášterský<SEP>Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00,1958-05-20,1958-05-20
118505,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ƶ,parsed:Ƶ,cleaned:ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00,1850-07-09,1850-07-09
91746,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen",parsed:S.P. Pinnerup<SEP>B. Ǿllgaard<SEP>L. Ho...,cleaned:S.P. Pinnerup<SEP>B. Ǿllgaard<SEP>L. H...,1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00,1975-08-29,1975-08-29
118508,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,В. Пертовский и Т. Королева. V. Petrovsky & T....,parsed:В.Пертовский и Т. Королева. V. Petrovsk...,cleaned:В.Пертовский и Т. Королева. V. Petrovs...,1,http://www.botanicalcollections.be/specimen/BR...,1973-06-24 00:00:00,1973-06-24,1973-06-24


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (18219 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
44231,A,Hameln,NaN,NaN,NaN,NaN,NaN,NaN,Hameln [a/d Weser],parsed:Hameln a<SEP>d Weser,cleaned:Hameln A<SEP>D Weser,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
64347,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,Lochenies in A. Brabant,parsed:Lochenies in A. Brabant,cleaned:Lochenies in A. Brabant,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00,1892-01-01,1892-01-01
34303,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,Fleury in A. Chevalier,parsed:Fleury in A. Chevalier,cleaned:Fleury in A. Chevalier,2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
22293,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN,Dandois in A. Cogniaux,parsed:Dandois in A. Cogniaux,cleaned:Dandois in A. Cogniaux,1,http://www.botanicalcollections.be/specimen/BR...,1866-06-01 00:00:00,1866-06-01,1866-06-01
39921,A. Cogniaux,Gilbert,NaN,in,NaN,NaN,NaN,NaN,Gilbert in A. Cogniaux,parsed:Gilbert in A. Cogniaux,cleaned:Gilbert in A. Cogniaux,2,http://www.botanicalcollections.be/specimen/BR...,1864-08-06 00:00:00,1864-08-06,1864-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95267,Čermak,Stan,NaN,NaN,NaN,NaN,NaN,NaN,Stan. Čermak,parsed:Stan Čermak,cleaned:Stan Čermak,1,http://www.botanicalcollections.be/specimen/BR...,1926-05-01 00:00:00,1926-05-01,1926-05-01
95262,Čermák,Stan,NaN,NaN,NaN,NaN,NaN,NaN,Stan Čermák,parsed:Stan Čermák,cleaned:Stan Čermák,1,http://www.botanicalcollections.be/specimen/BR...,1925-10-12 00:00:00,1925-10-12,1925-10-12
97476,Şes-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,Tatiana Şes[]-Frăţilescu,parsed:Tatiana Şes-Frăţilescu,cleaned:Tatiana Şes-Frăţilescu,1,http://www.botanicalcollections.be/specimen/BR...,1972-06-13 00:00:00,1972-06-13,1972-06-13
97477,Şesan-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,Tatiana Şesan-Frăţilescu,parsed:Tatiana Şesan-Frăţilescu,cleaned:Tatiana Şesan-Frăţilescu,9,http://www.botanicalcollections.be/specimen/BR...,1972-04-06 08:00:00,1971-07-12,1973-06-12


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))


----------------------------------------
show names with **particle** found 8304 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
12667,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN
64347,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN
34303,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN
49494,A. Chrková,J. Chrtek,NaN,at,NaN,NaN,NaN,NaN
22293,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 192 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
35113,Almeda,Frank,Jr.,NaN,NaN,NaN,NaN,NaN
69201,Anderson,W.A.,Jr.,NaN,NaN,NaN,NaN,NaN
52898,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN
61685,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN
35281,Barkley,Robert,II,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1131 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
34811,A. Hardy,Lebrun,NaN,in,NaN,NaN,Fr.,NaN
9316,A. Hardy,Lebrun,NaN,in,NaN,NaN,Fr,NaN
117554,Abbon,G.,NaN,NaN,NaN,NaN,fr.,NaN
109163,Ahlberg,NaN,NaN,NaN,NaN,NaN,Fr.,NaN
34755,Ahlfvengren,E.,NaN,NaN,NaN,NaN,Fr.,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given)
  )
)

# add collectors.particle if particle has multiple words, like “van der” or “Reyna de”
criterion = collectors["particle"].str.contains("\w+ \w+", na=False)

collectors['canonical_string_collector_parsed'][criterion] = collectors[criterion].apply(
    lambda this_df: "{particle} {family}, {given}".format(
        particle=this_df["particle"], 
        family=this_df["family"], 
        given=this_df["given"]
    ), axis="columns"
)
criterion = collectors["particle"].str.contains("\w+ \w+", na=False)
display(collectors[criterion].head())

/tmp/ipykernel_20331/1270537426.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors['canonical_string_collector_parsed'][criterion] = collectors[criterion].apply(


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max,canonical_string_collector_parsed
5114,A. Hardy,A.,NaN,coenen in,NaN,NaN,NaN,NaN,Abbé Strail & A. coenen in A. Hardy,parsed:Strail<SEP>A. coenen in A. Hardy,cleaned:Strail<SEP>A. coenen in A. Hardy,1,http://www.botanicalcollections.be/specimen/BR...,1868-01-01 00:00:00,1868-01-01,1868-01-01,"coenen in A. Hardy, A."
1670,A. Hardy,A. Hardy,NaN,avec Mr. Fonsny in,NaN,NaN,NaN,NaN,A. Hardy avec Mr. Fonsny in A. Hardy,parsed:A.Hardy avec Mr. Fonsny in A. Hardy,cleaned:A. Hardy avec Mr. Fonsny in A. Hardy,1,http://www.botanicalcollections.be/specimen/BR...,1883-07-23 00:00:00,1883-07-23,1883-07-23,"avec Mr. Fonsny in A. Hardy, A. Hardy"
109925,A. Hardy,H. Ponckier,NaN,in L. Quadvlieg in,NaN,NaN,NaN,NaN,[H. Ponckier] in L. Quadvlieg in A. Hardy,parsed:H.Ponckier in L. Quadvlieg in A. Hardy,cleaned:H. Ponckier in L. Quadvlieg in A. Hardy,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,"in L. Quadvlieg in A. Hardy, H. Ponckier"
60185,A. Hardy,L. Coututier,NaN,in Simon in,NaN,NaN,NaN,NaN,L. Coututier in Simon in A. Hardy,parsed:L.Coututier in Simon in A. Hardy,cleaned:L. Coututier in Simon in A. Hardy,1,http://www.botanicalcollections.be/specimen/BR...,1877-01-01 00:00:00,1877-01-01,1877-01-01,"in Simon in A. Hardy, L. Coututier"
62144,A. Hardy,Laboulle,NaN,de Verviers in,NaN,NaN,NaN,NaN,Laboulle de Verviers in A. Hardy,parsed:Laboulle de Verviers in A. Hardy,cleaned:Laboulle de Verviers in A. Hardy,1,http://www.botanicalcollections.be/specimen/BR...,1869-01-01 00:00:00,1869-01-01,1869-01-01,"de Verviers in A. Hardy, Laboulle"


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
58982,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,Klášterský & Žertová
118505,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,Ƶ
91746,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.","S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen"
118508,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,"Т. Королева. V. Petrovsky, В.Пертовский",В. Пертовский и Т. Королева. V. Petrovsky & T....
52117,Ẅttewaall,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Ẅttewaall, J.",J. [Ẅ]ttewaall


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Church, A.C., Ismail, Ruskandi, A",25,http://www.botanicalcollections.be/specimen/BR...,1930-04-21 21:33:20,1853-01-01,1995-10-26
1,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",Br. [Dedothaures] in A Robyns,1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00,1954-08-01,1954-08-01
2,A'd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'd,A'[D],1,http://www.botanicalcollections.be/specimen/BR...,1896-08-06 00:00:00,1896-08-06,1896-08-06
3,A'hlbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'hlbon,A'hlbon,1,http://www.botanicalcollections.be/specimen/BR...,1864-05-01 00:00:00,1864-05-01,1864-05-01
4,A,H. M. Llo,NaN,NaN,NaN,NaN,NaN,NaN,"A, H. M. Llo",H.M. L[lo][???]a,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69073,Žertová,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Žertová, A.",A. Žertová,20,http://www.botanicalcollections.be/specimen/BR...,1959-04-19 20:00:00,1958-07-09,1963-06-27
69074,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,Ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00,1850-07-09,1850-07-09
69075,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.","S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00,1975-08-29,1975-08-29
69076,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,"Т. Королева. V. Petrovsky, В.Пертовский",В. Пертовский и Т. Королева. V. Petrovsky & T....,1,http://www.botanicalcollections.be/specimen/BR...,1973-06-24 00:00:00,1973-06-24,1973-06-24


## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [9]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [10]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['A R', ' Ro', 'Rob', 'oby', 'byn', 'yns', 'ns ', 's B', ' Br']
- data from match-test: ['Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' O ', 'O H']
- data from match-test (full name): ['Tsy', 'syb', 'ybu', 'bul', 'uls', 'lsk', 'ska', 'kay', 'aya', 'ya ', 'a M', ' Ma', 'Mar', 'ari', 'ria', 'ia ', 'a P', ' Pa', 'Pav', 'avl', 'vlo', 'lov', 'ovn', 'vna', 'na ', 'a N', ' Na', 'Nag', 'agi', 'gib', 'ibi', 'bin', 'ina']


In [11]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))


(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(Lindberg), H.v.R. = (Lindberg), Hildur von Rettig'
'(Magistad), I.K. = (Magistad), Inger Kaasa'
'(Tsybulskaya), M.P.N. = (Tsybulskaya), Maria Pavlovna Nagibina'
'(botaniker), B.P. = (botaniker), Bror Pettersson'


In [12]:
def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

In [13]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
matches

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 6.541628122329712 s
Cossim matches calculated after 8.18178677558899 s
Get all matches together ...
Done. Matches calculated after 8.673712491989136 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,29472,"Rao, R.S.","Rao, R.S.",1.0
1,14852,"Hankenson, E.L.","Hankenson, E.L.",1.0
2,14861,"Hankó, B.","Hankó, B.",1.0
3,34252,"Stauffer, H.U.","Stauffer, H.U.",1.0
4,34249,"Staudt, A.","Staudt, A.",1.0
...,...,...,...,...
39548,18301,"Junr, J. Howse","Howse, J.",0.5
39549,18329,"Jusa, C.","Musa, A.",0.5
39550,12276,"G.-H. Parent, R. Turet","Parent, G.H.",0.5
39551,35129,"Tasker, S.","Taskin, E.",0.5


In [14]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

matches_fullnames

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 4.363322496414185 s
Cossim matches calculated after 6.107501983642578 s
Get all matches together ...
Done. Matches calculated after 6.1849586963653564 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,4930,"Mörner, Carl","Mörner, Carl",1.0
1,3492,"Jaramillo, Jaime","Jaramillo, Jaime",1.0
2,891,"Buchanan, John","Buchanan, John",1.0
3,3602,"Jones, Marcus E.","Jones, Marcus E",1.0
4,3599,"Jones, George Neville","Jones, George Neville",1.0
...,...,...,...,...
7223,3294,"Husain, Jardique","Husain, S S",0.5
7224,2089,"F. Crépin, Lennari","Crépin, François",0.5
7225,1166,"Chavarría, Ulises","Ferreira, Ulisses L C",0.5
7226,1099,"Chaboisseau. Ern. Malinvaud, AbB.","Chaboisseau, Théodore",0.5


### Create Output Results

Combine the matches data frame back to the (Meise) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [15]:
# # join (only) abbreviated name matches with collector source data
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True)
collectors_matches # 17552 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
1,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",Br. [Dedothaures] in A Robyns,1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00,1954-08-01,1954-08-01,0.0,"A Robyns, Br","Robyns, A.G.M.W.A.",0.575
7,A-J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A-J,A-J & Christ. Bange,1,http://www.botanicalcollections.be/specimen/BR...,1948-07-01 00:00:00,1948-07-01,1948-07-01,1.0,A-J,"Li, A.J.",0.551
8,A. Bauy Braun,A.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Bauy Braun, A.",A. B[auy] [= Braun A.?],1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,2.0,"A. Bauy Braun, A.","Braun, A.",0.551
12,A. Campos,V.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Campos, V.","Pedro Tenorio L., G. Toriz A., A. Campos V., O...",2,http://www.botanicalcollections.be/specimen/BR...,1987-03-01 00:00:00,1987-03-01,1987-03-01,3.0,"A. Campos, V.","Campos, M.M.",0.690
14,A. Chrková,J. Chrtek,NaN,at,NaN,NaN,NaN,NaN,"A. Chrková, J. Chrtek",J. Chrtek at A. Chrková,1,http://www.botanicalcollections.be/specimen/BR...,1976-07-27 00:00:00,1976-07-27,1976-07-27,4.0,"A. Chrková, J. Chrtek","Chrtek, J.",0.514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69069,Żytkowicz,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Żytkowicz, M.",M. Żytkowicz,6,http://www.botanicalcollections.be/specimen/BR...,2007-05-19 00:00:00,2007-05-19,2007-05-19,39548.0,"Żytkowicz, M.","Browicz, K.",0.568
69071,Željazova,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Željazova, B.",B. Željazova,2,http://www.botanicalcollections.be/specimen/BR...,1956-05-18 00:00:00,1955-05-28,1957-05-09,39549.0,"Željazova, B.","Sijazov, M.M.",0.508
69075,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.","S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00,1975-08-29,1975-08-29,39550.0,"Ǿllgaard, B.","Øllgaard, B.",1.000
69076,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,"Т. Королева. V. Petrovsky, В.Пертовский",В. Пертовский и Т. Королева. V. Petrovsky & T....,1,http://www.botanicalcollections.be/specimen/BR...,1973-06-24 00:00:00,1973-06-24,1973-06-24,39551.0,"Т. Королева. V. Petrovsky, В.Пертовский","Petrovsky, A.(.S.",0.700


In [16]:
# join (only) full name matches with collector source data
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname # 429 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,A,Hameln,NaN,NaN,NaN,NaN,NaN,NaN,"A, Hameln",Hameln [a/d Weser],1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,0,"A, Hameln","Hamel, A",0.533
1,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,"A. Chevalier, Fleury",Fleury in A. Chevalier,2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,1,"A. Chevalier, Fleury","Chevalier, Charles",0.533
2,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Dandois",Dandois in A. Cogniaux,1,http://www.botanicalcollections.be/specimen/BR...,1866-06-01 00:00:00,1866-06-01,1866-06-01,2,"A. Cogniaux, Dandois","Cogniaux, Alfred",0.600
3,A. Cogniaux,Gilbert,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Gilbert",Gilbert in A. Cogniaux,2,http://www.botanicalcollections.be/specimen/BR...,1864-08-06 00:00:00,1864-08-06,1864-08-06,3,"A. Cogniaux, Gilbert","Cogniaux, Alfred",0.618
4,A. Cogniaux,Guilmot,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Guilmot",Guilmot in A. Cogniaux,1,http://www.botanicalcollections.be/specimen/BR...,1867-05-19 00:00:00,1867-05-19,1867-05-19,4,"A. Cogniaux, Guilmot","Cogniaux, Alfred",0.586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7223,Øllgaard,Benjamin,NaN,NaN,NaN,NaN,NaN,NaN,"Øllgaard, Benjamin","Kai Larsen, Peder Pedersen & Benjamin Øllgaard",27,http://www.botanicalcollections.be/specimen/BR...,1968-02-01 20:10:18,1965-06-23,1996-07-15,7223,"Øllgaard, Benjamin","Øllgaard, Benjamin",1.000
7224,Øllgaard,Hans,NaN,NaN,NaN,NaN,NaN,NaN,"Øllgaard, Hans",Hans Øllgaard,1,http://www.botanicalcollections.be/specimen/BR...,1974-06-25 00:00:00,1974-06-25,1974-06-25,7224,"Øllgaard, Hans","Øllgaard, Hans",1.000
7225,Østrand,Eva,NaN,NaN,NaN,NaN,NaN,NaN,"Østrand, Eva",Eva Østrand,1,http://www.botanicalcollections.be/specimen/BR...,1939-07-20 00:00:00,1939-07-20,1939-07-20,7225,"Østrand, Eva","Strand, Elling Johan",0.534
7226,F. Crépin,Mepho Ma,NaN,à Lacoizqueta in,NaN,NaN,NaN,NaN,"à Lacoizqueta in F. Crépin, Mepho Ma",[Mepho Ma à] Lacoizqueta in F. Crépin,1,http://www.botanicalcollections.be/specimen/BR...,1881-05-23 00:00:00,1881-05-23,1881-05-23,7226,"à Lacoizqueta in F. Crépin, Mepho Ma","Lacoizqueta, José María de",0.551


In [17]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family', 'given'], ascending=[False, True, False], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
192,Aalto,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalto, M.",M. Aalto,16,http://www.botanicalcollections.be/specimen/BR...,1976-07-05 11:12:00,1975-07-07,1976-09-09,65.0,"Aalto, M.","Aalto, M.",1.0
199,Aas,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Aas, O.",Aas O.,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,69.0,"Aas, O.","Aas, O.",1.0
221,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, A.",A. Abbas,44,http://www.botanicalcollections.be/specimen/BR...,1934-10-15 01:39:19,1869-05-04,2001-08-10,78.0,"Abbas, A.","Abbas, A.",1.0
226,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.","H.M. Raup, E.C. Abbe",1,http://www.botanicalcollections.be/specimen/BR...,1932-09-01 00:00:00,1932-09-01,1932-09-01,81.0,"Abbe, E.C.","Abbe, E.C.",1.0
237,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",W.L. Abbott,1,http://www.botanicalcollections.be/specimen/BR...,1922-02-26 00:00:00,1922-02-26,1922-02-26,90.0,"Abbott, W.L.","Abbott, W.L.",1.0


Save the plain name matching results only ...

In [18]:
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231030
# 
# this_output_file='data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

In [19]:
# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [20]:
# merge now with WikiData: the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [21]:
print("Show some name match examples (e.g. «Louis…» matching various names) …")
for testname in ['Louis', 'Abbot']:
    criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith(testname))    
    this_table=collectors_matches_g1_merged_wikidata[criterion].get([
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        # 'canonical_string_fullname', 
        'itemLabel', 'wikidata_link',
        'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod', 'wyb', 'wye'
    ]).sort_values(by=['namematch_similarity'], ascending=[False])
    print("# ---------------------------------------------\n# «%s…» as test name, %d collector names begin with:" % (testname, criterion.sum()))    
    display(this_table)

Show some name match examples (e.g. «Louis…» matching various names) …
# ---------------------------------------------
# «Louis…» as test name, 68 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
22948,9,http://www.botanicalcollections.be/specimen/BR...,Louis-Marie,Louis-Marie,1.000,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1934-08-03,1952-07-01,1896,1978,<NA>,<NA>
27663,2348,http://www.botanicalcollections.be/specimen/BR...,"Louis, A.M.","Louis, A.M.",1.000,Adriaan M. Louis,http://www.wikidata.org/wiki/Q21338327,1902-11-28,1995-05-04,1944,<NA>,<NA>,<NA>
22950,22,http://www.botanicalcollections.be/specimen/BR...,"Louis-Marie, P.",Louis-Marie,0.914,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1921-08-19,1935-08-07,1896,1978,<NA>,<NA>
19944,30727,http://www.botanicalcollections.be/specimen/BR...,"Louis, J.","Louis, A.",0.877,A. Louis,http://www.wikidata.org/wiki/Q33682458,1875-08-01,1987-08-29,<NA>,<NA>,<NA>,<NA>
19946,2,http://www.botanicalcollections.be/specimen/BR...,"Louis, M.","Louis, A.",0.870,A. Louis,http://www.wikidata.org/wiki/Q33682458,1930-09-01,1994-09-22,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47049,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Charlet","Pinel, Charles",0.508,Charles Pinel,http://www.wikidata.org/wiki/Q21523018,1913-07-01,1913-07-01,1802,1871,<NA>,<NA>
22937,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, H. Leters",Louis-Marie,0.508,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1905-06-27,1905-06-27,1896,1978,<NA>,<NA>
22931,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, A. Bodart",Louis-Marie,0.502,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1921-05-29,1921-05-29,1896,1978,<NA>,<NA>
22939,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, J. Hassart",Louis-Marie,0.500,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1913-06-08,1913-06-08,1896,1978,<NA>,<NA>


# ---------------------------------------------
# «Abbot…» as test name, 6 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
434,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, A.T.D.","Abbott, A.T.D.",1.000,A. T. D. Abbott,http://www.wikidata.org/wiki/Q117328147,2010-01-07,2010-01-07,1936,2013,<NA>,<NA>
439,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, W.L.","Abbott, W.L.",1.000,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26,1922-02-26,1860,1936,<NA>,<NA>
436,2,http://www.botanicalcollections.be/specimen/BR...,"Abbott, R.","Abbott, R.J.",0.910,R.J. Abbott,http://www.wikidata.org/wiki/Q33660683,1995-01-27,1995-06-20,<NA>,<NA>,<NA>,<NA>
433,2,http://www.botanicalcollections.be/specimen/BR...,Abbott,"Abbott, G.",0.867,George Abbott,http://www.wikidata.org/wiki/Q47112598,1982-11-16,1982-11-16,<NA>,<NA>,<NA>,<NA>
438,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, W.C.","Abbott, W.L.",0.861,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,NaT,NaT,1860,1936,<NA>,<NA>
435,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, J. Richard","Abbott, J.R.",0.765,J. Richard Abbott,http://www.wikidata.org/wiki/Q18982386,1995-03-22,1995-03-22,1968,<NA>,<NA>,<NA>


In [22]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [23]:
collectors_matches_g1_merged_wikidata.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",Br. [Dedothaures] in A Robyns,...,Q5675465,1935,2003,<NA>,<NA>,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
1,Robyns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Robyns,Robyns?,...,Q5675465,1935,2003,<NA>,<NA>,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
2,Robyns,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Robyns, A.",W. et A. Robyns,...,Q5675465,1935,2003,<NA>,<NA>,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
3,Robyns,A.G.,NaN,NaN,NaN,NaN,NaN,NaN,"Robyns, A.G.","W.H. Lewis, J.L. Hawker, R.L. Oliver, A.G. Rob...",...,Q5675465,1935,2003,<NA>,<NA>,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
4,Robyns,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Robyns, D.",D. Robyns & Lebrun J.,...,Q5675465,1935,2003,<NA>,<NA>,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465


In [24]:
# Select useful columns for data results
collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family', 'given'], ascending=[False, True, True], inplace=True)

collectors_wikidata_cossim # comparison-match of «Kotschy, Karl Georg Th» (collector data) →← «Kotschy, T» (Wikidata) has only 0.5 similarity but corresponds to the correct person name we need

/tmp/ipykernel_20331/3796250127.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family', 'given'], ascending=[False, True, True], inplace=True)


,canonical_string_collector_parsed,family,given,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,source_data,namematch_source_data,namematch_resource_data,namematch_similarity,item,...,harv,ipni,abbr,bionomia_id,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb
395,"Aalto, M.",Aalto,M.,16,http://www.botanicalcollections.be/specimen/BR...,M. Aalto,"Aalto, M.","Aalto, M.",1.0,http://www.wikidata.org/entity/Q964994,...,6655,30583-1,Aalto,NaN,1976-07-05 11:12:00,1975-07-07,1976-09-09,1939,<NA>,<NA>
406,"Aas, O.",Aas,O.,1,http://www.botanicalcollections.be/specimen/BR...,Aas O.,"Aas, O.","Aas, O.",1.0,http://www.wikidata.org/entity/Q36646127,...,74987,30584-1,Aas,NaN,NaT,NaT,NaT,<NA>,<NA>,<NA>
417,"Abbas, A.",Abbas,A.,44,http://www.botanicalcollections.be/specimen/BR...,A. Abbas,"Abbas, A.","Abbas, A.",1.0,http://www.wikidata.org/entity/Q60141229,...,NaN,20034668-1,Al.Abbas,NaN,1934-10-15 01:39:19,1869-05-04,2001-08-10,<NA>,<NA>,<NA>
418,"Abbas, A.",Abbas,A.,44,http://www.botanicalcollections.be/specimen/BR...,A. Abbas,"Abbas, A.","Abbas, A.",1.0,http://www.wikidata.org/entity/Q88804360,...,NaN,20034420-1,A.Abbas,NaN,1934-10-15 01:39:19,1869-05-04,2001-08-10,<NA>,<NA>,<NA>
429,"Abbe, E.C.",Abbe,E.C.,1,http://www.botanicalcollections.be/specimen/BR...,"H.M. Raup, E.C. Abbe","Abbe, E.C.","Abbe, E.C.",1.0,http://www.wikidata.org/entity/Q10274118,...,30066,26-1,Abbe,Q10274118,1932-09-01 00:00:00,1932-09-01,1932-09-01,1905,2000,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,"Vedia, J.",Vedia,J.,2,http://www.botanicalcollections.be/specimen/BR...,"A. Lliully., R. Léon., A. Ortega., J. Vedia","Vedia, J.","Heredia, G.",0.5,http://www.wikidata.org/entity/Q5873723,...,42416,20000963-1,Heredia,NaN,2005-10-18 00:00:00,2005-10-17,2005-10-19,1959,<NA>,<NA>
40533,"Wachiori, P.",Wachiori,P.,1,http://www.botanicalcollections.be/specimen/BR...,P. Wachiori,"Wachiori, P.","Marchiori, J.N.C.",0.5,http://www.wikidata.org/entity/Q22708778,...,88834,20008955-1,Marchiori,NaN,1972-11-15 00:00:00,1972-11-15,1972-11-15,<NA>,<NA>,<NA>
40623,"Walderen, W.",Walderen,W.,1,http://www.botanicalcollections.be/specimen/BR...,W. Van Walderen,"Walderen, W.","Walden, G.K.",0.5,http://www.wikidata.org/entity/Q15975611,...,100412,20017731-1,Walden,NaN,1884-06-12 00:00:00,1884-06-12,1884-06-12,<NA>,<NA>,<NA>
41097,"Whitcher, Nd C.",Whitcher,Nd C.,1,http://www.botanicalcollections.be/specimen/BR...,[ND] C. Whitcher,"Whitcher, Nd C.","Pitcher, Z.",0.5,http://www.wikidata.org/entity/Q8072235,...,11946,7733-1,Pitcher,Q8072235,NaT,NaT,NaT,1797,1872,<NA>


In [25]:
# # TODO further evaluation or filtering, counting, clean up aso.
# if not os.path.exists('data'):
#     os.makedirs('data')
# 
# # meise_collectors_cosine-similarity_wikidata-botanists_%s.csv
# this_output_file='data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_wikidata_cossim.to_csv(this_output_file)
# 
# print("Wrote matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

## Output Mapping to DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

## Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [26]:
pprint.pprint(collectors_wikidata_cossim.columns)

Index(['canonical_string_collector_parsed', 'family', 'given',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'source_data', 'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'canonical_string', 'itemLabel',
       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id',
       'collectors_eventDate_mean', 'collectors_eventDate_min',
       'collectors_eventDate_max', 'yob', 'yod', 'wyb'],
      dtype='object')


In [27]:
# TODO add score (similarity + ((yob,yod) ~ (eventDate_min, eventDate_max))

collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(
    by=['namematch_similarity', 'family', 'given'], 
    ascending=[False, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossim.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy()

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_similarity_annotation'] = dwcagent_attr_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 0

# False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -1
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = 0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime"
] = 0.5

# False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime"
] = -0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -0.5

# annotations True cases
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "full match"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "OK? year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "OK? year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "unknown life time"

# annotations False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

dwcagent_attr_output['custom_score_overall'] = (
    dwcagent_attr_output['namematch_similarity'] * \
    (
        ( dwcagent_attr_output["custom_score_lifetime"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime"],
    lifetime_score_annote=row["custom_score_lifetime_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    by=['namematch_similarity', 'family', 'given', 'custom_score_overall'], 
    ascending=[False, True, True, False], 
    inplace=True
)

# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
show_display_output=True
if show_display_output:
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))

/tmp/ipykernel_20331/1341979843.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
395,M. Aalto,Marjatta Aalto,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1939-?,1975-07-07,1976-09-09,True,<NA>,1.0,OK? year of death is missing
429,E.C. Abbe,Ernst Cleveland Abbe,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1905-2000,1932-09-01,1932-09-01,True,True,1.0,full match
434,A.T.D. Abbott,A. T. D. Abbott,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1936-2013,2010-01-07,2010-01-07,True,True,1.0,full match
439,W.L. Abbott,William Louis Abbott,0.5,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.0,1860-1936,1922-02-26,1922-02-26,True,True,1.0,full match
479,J.E.C. Aberdeen,John Errol Chandos Aberdeen,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1913-1996,1961-05-04,1964-03-31,True,True,1.0,full match


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
648,B.R. Adams,Bryan Roger Adams,-0.25,1.0 (cos similarity); -0.25 (score overall); 1...,0.0,1.0,1942-?,1872-01-01,1979-05-12,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
758,A. Afzelius,Adam Afzelius,0.25,1.0 (cos similarity); 0.25 (score overall); 17...,0.0,1.0,1750-1837,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
762,P.C. Afzelius,Pehr Conrad Afzelius,0.00,1.0 (cos similarity); 0.00 (score overall); 18...,-0.5,1.0,1817-1850,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
809,R. Aguilar,Reinaldo Aguilar,-0.25,1.0 (cos similarity); -0.25 (score overall); 1...,0.0,1.0,1962-?,1945-01-11,2003-11-19,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
861,S. Ahmad,Shamsuddin Ahmad,-0.50,1.0 (cos similarity); -0.50 (score overall); 1...,-0.5,1.0,1942-?,1948-08-01,1976-08-18,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."


In [28]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_similarity':               'custom_namematch_similarity'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    dwcagent_attr_output.head(20)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]


In [29]:
show_display_output=True
if show_display_output:
    criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('Louis'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda x: x < 0 )
    
    display(dwcagent_attr_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime
417,http://www.botanicalcollections.be/specimen/BR...,A. Abbas,A. Abbas,1,Alia Abbas,1.0 (cos similarity); -0.25 (score overall); ?...,1869-05-04,2001-08-10,Person,collected,wikidata,http://www.wikidata.org/entity/Q60141229,-0.25,1.0,-0.5,0.0
418,http://www.botanicalcollections.be/specimen/BR...,A. Abbas,A. Abbas,2,Abdulla Abbas,1.0 (cos similarity); -0.25 (score overall); ?...,1869-05-04,2001-08-10,Person,collected,wikidata,http://www.wikidata.org/entity/Q88804360,-0.25,1.0,-0.5,0.0
544,http://www.botanicalcollections.be/specimen/BR...,M. Ackermann & Ben Böttcher,M. Ackermann,1,Markus Ackermann,1.0 (cos similarity); 0.25 (score overall); 19...,2003-06-20,2003-06-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.25,1.0,-0.5,1.0
545,http://www.botanicalcollections.be/specimen/BR...,M. Ackermann & Ben Böttcher,M. Ackermann,2,Marianne Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2003-06-20,2003-06-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,1.0,-0.5,0.0
546,http://www.botanicalcollections.be/specimen/BR...,M. Ackermann & Ben Böttcher,M. Ackermann,3,Manfred Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2003-06-20,2003-06-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,1.0,-0.5,0.0
690,http://www.botanicalcollections.be/specimen/BR...,Laurence O. and M.K. Adhikari,M.K. Adhikari,1,Mahesh Kumar Adhikari,1.0 (cos similarity); -0.25 (score overall); ?...,2001-05-01,2001-05-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q10323105,-0.25,1.0,-0.5,0.0
691,http://www.botanicalcollections.be/specimen/BR...,Laurence O. and M.K. Adhikari,M.K. Adhikari,2,Mahesh Kumar Adhikari,1.0 (cos similarity); -0.25 (score overall); ?...,2001-05-01,2001-05-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112706,-0.25,1.0,-0.5,0.0
762,http://www.botanicalcollections.be/specimen/BR...,P.C. Afzelius,P.C. Afzelius,1,Pehr Conrad Afzelius,1.0 (cos similarity); 0.00 (score overall); 18...,NaT,NaT,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504600,0.00,1.0,-0.5,0.5
761,http://www.botanicalcollections.be/specimen/BR...,P.C. Afzelius,P.C. Afzelius,2,Pehr Conrad Afzelius,1.0 (cos similarity); -0.25 (score overall); ?...,NaT,NaT,Person,collected,wikidata,http://www.wikidata.org/entity/Q10347935,-0.25,1.0,-0.5,0.0
817,http://www.botanicalcollections.be/specimen/BR...,Sipman H. & J. Aguirre,J. Aguirre,1,Jaime Aguirre,1.0 (cos similarity); 0.25 (score overall); 19...,1982-08-18,1988-10-25,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504658,0.25,1.0,-0.5,1.0


In [30]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20231030

this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_meise_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20231030.csv (18845 kB)


## Documentation

Explanation of columns (TODO review documentation):

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | the first date of eventDate (supposedly the first sampling date), but grouped from collector name (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.)—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | the last date of eventDate (supposedly the last sampling date), but grouped from collector name (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.)—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))